In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
  

"""
fe : ratio of ethanol 
fpg : ratio of PG
Cm : experimental data of solubiluty in mixter based on ratio
Cw : solubility in pure water
Cpg : solubility in pure PG
Ce : solubility in pure ethanol
T : temperature
LCm,LCe, ... : log of Cm, ...

x , y : for finding j with regression

Sm : prediction data of solubility 
LSm : prediction data of log Sm

"""

# Binary mixture

In [ ]:
# function for change molar to mole fraction
# density of solvent
de = 0.785
dw = 0.997
dpg = 1.0273
    # solvent Molecular weight
mw_w = 18
mw_e = 46
mw_pg = 76.09
def Mole_fraction (M, d1, d2, mw_1, mw_2 ):
    if len(M) == 7 :
        f1 = [1,0.8,0.6,0.5,0.4,0.2,0]
        f2 = [0,0.2,0.4,0.5,0.6,0.8,1]
        f1 = np.array(f1).reshape(-1,1)
        f2= np .array(f2).reshape(-1,1)
    else:
        f1 = np.arange(0,1.01, 0.01)
        f2 = 1-f1
    
    mol_1 = (f1*d1)/mw_1
    mol_2 = (f2*d2)/mw_2
    X2 = mol_2/(mol_2+mol_1)
    X1 = mol_1/(mol_2+mol_1)
    MW = X2*mw_2 + X1*mw_1
    d = (f1*d1) + (f2*d2)
    m = (M/(1000*d))*MW
    Xe = m/(m+1)
    return (Xe, MW, d)
    #print('exp mole fraction = ' , Xe)


   


####  EHSA function for binary mixture

def EHSA (exp_data, T_fusion, dH_fusion, D_SP, D_V, sp1, sp2, d1, d2, mw_1, mw_2):
    
    Xe, MW, d = Mole_fraction(exp_data, d1, d2, mw_1, mw_2)
    f1 = [1,0.8,0.6,0.5,0.4,0.2,0]
    f2 = [0,0.2,0.4,0.5,0.6,0.8,1]
    f1 = np.array(f1).reshape(-1,1)
    f2= np .array(f2).reshape(-1,1)
    
    spmix = (f1*sp1) + (f2*sp2)
    
    # T0 : heat fusion temp
    T0 = T_fusion
    # dF : heat of fusion cal/mole
    dHf = dH_fusion
    # sp = drug Solubilitty parameter (cal/cm3)^0.5
    sp = D_SP
    # V2 = drug molar volume (cm3/mole)
    V2 = D_V
    
    # solvent molar volume (cm3/mole)
    V1 =  MW/d
    
    # volume fraction
    Q = 1
    #Q = (V1*(1-Xe))/((V1*(1-Xe)) + V2*Xe)
    #print ('Q', Q)
    
    # A coeficent
    A = (V2*Q)/(R*T)

    LXi = (dHf/(2.303*R*T)) * ((T0-T)/T0)
    #LXi = (dHf/(2.303*R*T)) * (math.log10(T0/T))
    Xi = 10**(-LXi)
    # ideal solubility
    # calculate the W_exp
    W = ((np.log10(Xi/Xe)/A) + sp**2 + spmix**2)*0.5
    
    #### polynomial regression for EHSA ####
    # Importing the dataset
    ## x = data, y = quadratic equation
    x = spmix
    x1 = x.reshape(-1, 1)
    y = W
    '''
    plt.scatter(x, y, s = 10)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Non Linear Data')

    '''
    poly_features = PolynomialFeatures(degree = 2, include_bias = False)
    x_poly = poly_features.fit_transform(x1)
    lin_reg = LinearRegression()
    lin_reg.fit(x_poly, y)
    #print('Coefficients of x are', lin_reg.coef_)
    #print('Intercept is', lin_reg.intercept_)

    y_deg2 = lin_reg.predict(x_poly)
    # model evaluation
    mse_deg2 = mean_squared_error(y, y_deg2)

    r2_deg2 = r2_score(y, y_deg2)

    # printing values
    #print('MSE of Polyregression model', mse_deg2)
    #print('R2 score of Linear model: ', r2_deg2)

    # calculation of W_calc , X_calc
    Q2 = 1
    f12 = np.arange(0,1.01, 0.01)
    f22 = 1-f12
    spmix2 = (f22*sp2) + (f12*sp1)

    A2 = (V2*Q2)/(R*T)
    mol_2 = (f22*d2)/mw_2
    mol_1 = (f12*d1)/mw_1
    X2 = mol_2/(mol_2+mol_1)
    X1 = mol_1/(mol_2+mol_1)
    MW = X2*mw_2 + X1*mw_1
    d = (f22*d2) + (f12*d1)
    V1 =  MW/d

    for i in range (1):
        Q3 = Q2
        W_calc =  lin_reg.intercept_[0] + (spmix2 * lin_reg.coef_[0][0]) + ((spmix2**2)* lin_reg.coef_[0][1]) 

        LX_calc = LXi - A2*((sp**2) + (spmix2**2) - (2 * W_calc))

        X_calc = 10**-LX_calc

        #X_calc = np.array(X_calc).reshape(-1,1)

        Q2 = (V1*(1-X_calc))/((V1*(1-X_calc)) + V2*X_calc)
        A2 = (V2*Q2)/(R*T)


    #print ('Q2', Q2)
    #print ('A2', A2)
    #print("W_calc =", lin_reg.intercept_[0], '+' ,  lin_reg.coef_[0][0], "*S"  , "+" , lin_reg.coef_[0][1],  "*S^2 ")

    #print ('W_exp =' , W)
    #print ("W_calc =", W_calc)
    #print ('X_calc = ',X_calc )
    return (X_calc, spmix, spmix2)




# Ternary mixtures

In [ ]:

# function for change molar to mole fraction
# density of solvent
de = 0.785
dw = 0.997
dpg = 1.0273
    # solvent Molecular weight
mw_w = 18
mw_e = 46
mw_pg = 76.09
def Mole_fraction (M, d1, d2, d3, mw_1, mw_2, mw_3 ):
    if len(M) <= 10 :
        f1 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
        f2 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
        f3 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
        f1 = np.array(f1).reshape(-1,1)
        f2= np .array(f2).reshape(-1,1)
        f3= np .array(f3).reshape(-1,1)

    else:
        f1 = np.arange(0.1,0.8, 0.01)
        f2 = (1-f1)/2
        f3 = (1-f1)/2
    
    mol_1 = (f1*d1)/mw_1
    mol_2 = (f2*d2)/mw_2
    mol_3 = (f3*d3)/mw_3    
    X1 = mol_1/(mol_2+mol_1+mol_3)
    X2 = mol_2/(mol_2+mol_1+mol_3)
    X3 = mol_3/(mol_2+mol_1+mol_3)
    MW = X1*mw_1 + X2*mw_2 + X3*mw_3
    d = (f1*d1) + (f2*d2) + (f3*d3)
    m = (M/(1000*d))*MW
    Xe = m/(m+1)
    return (Xe, MW, d)
    #print('exp mole fraction = ' , Xe)


#### function EHSA

def EHSA (exp_data, T_fusion, dH_fusion, D_SP, D_V, sp1, sp2, sp3, d1, d2, d3, mw_1, mw_2, mw_3):
    
    Xe, MW, d = Mole_fraction(exp_data, d1, d2, d3, mw_1, mw_2, mw_3)
    
    f1 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
    f2 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
    f3 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
    f1 = np.array(f1).reshape(-1,1)
    f2= np .array(f2).reshape(-1,1)
    f3= np .array(f3).reshape(-1,1)
    
    
    spmix = (f1*sp1) + (f2*sp2) + (f3*sp3)
    
    # T0 : heat fusion temp
    T0 = T_fusion
    # dF : heat of fusion cal/mole
    dHf = dH_fusion
    # sp = drug Solubilitty parameter (cal/cm3)^0.5
    sp = D_SP
    # V2 = drug molar volume (cm3/mole)
    V2 = D_V
    
    # solvent molar volume (cm3/mole)
    V1 =  MW/d
    
    # volume fraction
    Q = 1
    #Q = (V1*(1-Xe))/((V1*(1-Xe)) + V2*Xe)
    #print ('Q', Q)
    
    # A coeficent
    A = (V2*Q)/(R*T)

    LXi = (dHf/(2.303*R*T)) * ((T0-T)/T0)
    #LXi = (dHf/(2.303*R*T)) * (math.log10(T0/T))
    Xi = 10**(-LXi)
    # ideal solubility
    # calculate the W_exp
    W = ((np.log10(Xi/Xe)/A) + sp**2 + spmix**2)*0.5
    
    #### polynomial regression for EHSA ####
    # Importing the dataset
    ## x = data, y = quadratic equation
    x = spmix
    x1 = x.reshape(-1, 1)
    y = W
    '''
    plt.scatter(x, y, s = 10)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Non Linear Data')

    '''
    poly_features = PolynomialFeatures(degree = 2, include_bias = False)
    x_poly = poly_features.fit_transform(x1)
    lin_reg = LinearRegression()
    lin_reg.fit(x_poly, y)
    #print('Coefficients of x are', lin_reg.coef_)
    #print('Intercept is', lin_reg.intercept_)

    y_deg2 = lin_reg.predict(x_poly)
    # model evaluation
    mse_deg2 = mean_squared_error(y, y_deg2)

    r2_deg2 = r2_score(y, y_deg2)

    # printing values
    #print('MSE of Polyregression model', mse_deg2)
    #print('R2 score of Linear model: ', r2_deg2)

    # calculation of W_calc , X_calc
    Q2 = 1
    #f12 = np.arange(0,1.01, 0.01)
    #f22 = 1-f12
    f12 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
    f22 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
    f32 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
    f12 = np.array(f12).reshape(-1,1)
    f22= np .array(f22).reshape(-1,1)
    f32= np .array(f32).reshape(-1,1)
    
    f12 = np.arange(0.1,0.8, 0.01)
    f22 = (1-f12)/2
    f32 = (1-f12)/2
    spmix2 = (f12*sp1) + (f22*sp2) + (f32*sp3)
    
    
    
    A2 = (V2*Q2)/(R*T)
    
    mol_1 = (f12*d1)/mw_1
    mol_2 = (f22*d2)/mw_2
    mol_3 = (f32*d3)/mw_3    
    X1 = mol_1/(mol_2+mol_1+mol_3)
    X2 = mol_2/(mol_2+mol_1+mol_3)
    X3 = mol_3/(mol_2+mol_1+mol_3)
    MW = X1*mw_1 + X2*mw_2 + X3*mw_3
    d = (f12*d1) + (f22*d2) + (f32*d3)

    
    V1 =  MW/d

    for i in range (1):
        Q3 = Q2
        W_calc =  lin_reg.intercept_[0] + (spmix2 * lin_reg.coef_[0][0]) + ((spmix2**2)* lin_reg.coef_[0][1]) 

        LX_calc = LXi - A2*((sp**2) + (spmix2**2) - (2 * W_calc))

        X_calc = 10**-LX_calc

        #X_calc = np.array(X_calc).reshape(-1,1)

        Q2 = (V1*(1-X_calc))/((V1*(1-X_calc)) + V2*X_calc)
        A2 = (V2*Q2)/(R*T)


    #print ('Q2', Q2)
    #print ('A2', A2)
    #print("W_calc =", lin_reg.intercept_[0], '+' ,  lin_reg.coef_[0][0], "*S"  , "+" , lin_reg.coef_[0][1],  "*S^2 ")

    #print ('W_exp =' , W)
    #print ("W_calc =", W_calc)
    #print ('X_calc = ',X_calc )
    return (X_calc, spmix, spmix2)
    